In [12]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

# 使用keras内置的mnist数据集
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
# 处理一下数据
training_images = training_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)
training_images, test_images = training_images / 255.0, test_images / 255.0
new_model = tf.keras.models.load_model('./weights/my_model.h5')
start = time.time()
new_model.evaluate(test_images,  test_labels)
print(time.time()-start)
#converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
#tflite_model = converter.convert()
#open("weights.tflite", "wb").write(tflite_model)



10000/10000 [==============================] - 1s 108us/sample - loss: 0.0335 - acc: 0.9908
1.0955400466918945


In [9]:
new_model.evaluate(test_images,  test_labels)
new_model.evaluate(training_images,  training_labels)

60000/60000 [==============================] - 6s 96us/sample - loss: 0.1165 - acc: 0.9665


[0.11654235598130229, 0.96646667]

In [2]:
import tensorflow as tf
version = tf.__version__
print(tf.config.list_physical_devices('GPU'))

[]


In [22]:
test = np.load("cov28_28_5_5_6.npy")

In [25]:
np.save("test_images.npy",test_images)
np.save("test_labels.npy",test_labels)

In [2]:
import numpy as np
weights = new_model.weights

In [5]:
cov28_28_5_5_6 = weights[0].numpy()
cov28_28_5_5_6.shape

(5, 5, 1, 6)

In [1]:
#检测代码原型
import numpy as np
weights = new_model.weights
#图片序号
num =0
#图片标签
print(test_labels[num])
input = tf.convert_to_tensor(test_images[num].reshape(-1,28,28,1),dtype="float32")
input = input/255
cov28_28_5_5_6 = tf.convert_to_tensor(weights[0].numpy())
cnn1 = tf.nn.conv2d(input,cov28_28_5_5_6,strides = [1,1,1,1], padding = 'VALID')  + weights[1]
signCnn1 = tf.tanh(cnn1)
poolCnn1 = tf.nn.max_pool(signCnn1,[1,2,2,1],[1,2,2,1],padding='VALID')
print(poolCnn1.shape)
cov12_12_5_5_6 = tf.convert_to_tensor(weights[2].numpy())
print(cov12_12_5_5_6.shape)
cnn2 = tf.nn.conv2d(poolCnn1,cov12_12_5_5_6,strides = [1,1,1,1], padding = 'VALID')  + weights[3]
signCnn2 = tf.tanh(cnn2)
poolCnn2 = tf.nn.max_pool(signCnn2,[1,2,2,1],[1,2,2,1],padding='VALID')
poolCnn2 = tf.reshape(poolCnn2,shape=[1,-1])
fn1 = tf.tanh(tf.matmul(poolCnn2, tf.convert_to_tensor(weights[4]))+weights[5])

print(fn1.shape)
fn2 = tf.tanh(tf.matmul(fn1, tf.convert_to_tensor(weights[6]))+weights[7])
print(fn2.shape)
print((tf.matmul(fn2, tf.convert_to_tensor(weights[8]))+weights[9]) * 255 * 255)
result = tf.nn.softmax(tf.matmul(fn2, tf.convert_to_tensor(weights[8]))+weights[9])
print(result.shape)
print(result)
print(tf.argmax(result,1))

NameError: name 'new_model' is not defined

In [9]:
#检测原型，浮点数转整数
import numpy as np
weights = new_model.weights
def simpletanh(data):
#    r = data/255/255
    r = data/256
    r = tf.maximum(r,-255)
    r = tf.minimum(r,255)
 #   r = tf.tanh(r) * 255
    r = tf.cast(r,tf.int32)
    return r
#图片序号
num = 7
truenum = 0
for i in range(1):
#图片标签
    i=7
    real = test_labels[i]
    input = tf.convert_to_tensor(test_images[i].reshape(-1,28,28,1),dtype="int32")
    cov28_28_5_5_6 = tf.convert_to_tensor((weights[0].numpy()*255).astype("int32"))
    cnn1 = tf.nn.conv2d(input,cov28_28_5_5_6,strides = [1,1,1,1], padding = 'VALID')  + (weights[1]*255*255).numpy().astype("int32")
    #print((weights[1]*255*255).numpy().astype("int32"))
    #print(cnn1[0,:,:,0])
    signCnn1 = simpletanh(cnn1)
    poolCnn1 = tf.nn.max_pool(signCnn1,[1,2,2,1],[1,2,2,1],padding='VALID')
    #print(poolCnn1)
    cov12_12_5_5_6 = tf.convert_to_tensor((weights[2].numpy()*255).astype("int32"))
    cnn2 = tf.nn.conv2d(poolCnn1,cov12_12_5_5_6,strides = [1,1,1,1], padding = 'VALID')  + (weights[3]*255*255).numpy().astype("int32")
    #print((weights[3]*255*255).numpy().astype("int32"))
    signCnn2 = simpletanh(cnn2)
    #print(signCnn2)
    print(signCnn2.shape)
    poolCnn2 = tf.nn.max_pool(signCnn2,[1,2,2,1],[1,2,2,1],padding='VALID')
    poolCnn2 = tf.reshape(poolCnn2,shape=[1,-1])
    print(poolCnn2)
    tmp = tf.matmul(poolCnn2, tf.convert_to_tensor((weights[4]*255).numpy().astype("int32"))) + (weights[5]*255*255).numpy().astype("int32").reshape(1,-1)
    #print(tf.convert_to_tensor((weights[4]*255).numpy().astype("int32")))
    #print((weights[5]*255).numpy().astype("int32").reshape(1,-1))

    fn1 = simpletanh(tmp)
    print(fn1)
    print(fn1.shape)
    tmp = tf.matmul(fn1, tf.convert_to_tensor((weights[6]*255).numpy().astype("int32"))) + (weights[7]*255*255).numpy().astype("int32").reshape(1,-1)
    fn2 = simpletanh(tmp)
    #print(fn2.shape)
    tmp = tf.matmul(fn2, tf.convert_to_tensor((weights[8]*255).numpy().astype("int32"))) + (weights[9]*255*255).numpy().astype("int32").reshape(1,-1)
    print(tmp)

    #print(tmp.shape)
    result = tf.argmax(tmp,1).numpy()[0]
    print(real,result)
    if(real == result):
        truenum = truenum + 1
    print(truenum/1000.0,end = '\r',flush=True)
    

(1, 8, 8, 16)
tf.Tensor(
[[-255 -255  255 -255  255 -255  255 -255 -255    3  255 -125  231 -255
   190 -123 -255 -255  255  255 -255 -255   78  255  255 -255  255 -170
   255  255  255 -255 -255  255 -255  255 -255  255 -116  201  255 -255
     7 -255  255  255 -166  -62  255  255 -255 -255   21   -4 -255 -120
  -255   95 -255 -255 -104  255 -255 -255 -255 -255  -27  255 -255   94
  -117 -255  255 -255 -255  255 -255 -255 -255 -255 -255 -172  -36  -64
  -255  255  255  167  255 -255  255  255  254 -255  255  195 -255 -255
    30   63  200  255  109   97  255 -156  255 -255  255  255  104 -255
  -255   30 -255 -255  201  255 -255 -240  189 -255 -243 -247   96  255
  -255  -62  255  255   71  255 -255 -205 -105 -255  179   39   90  255
  -255 -255 -137 -255  255 -125  255  255 -255   15 -223 -255  255  255
   151  255  -70 -255  255 -255 -255 -255 -255  236 -212 -255 -223 -187
   139  255 -240 -255 -255 -255 -255 -148 -255  -60 -255 -109 -255  255
  -255  -74  255 -170 -234 -149 -255   

In [9]:
np.savez("weights.npz",cov1 = (weights[0].numpy()*255).astype("int32"),
         cov1_b = (weights[1].numpy()*255*255).astype("int32"),
         cov2 = (weights[2].numpy()*255).astype("int32"),
         cov2_b = (weights[3].numpy()*255*255).astype("int32"),
         fn1 = (weights[4].numpy()*255).astype("int32"),
         fn1_b = (weights[5].numpy()*255*255).astype("int32"),
         fn2 = (weights[6].numpy()*255).astype("int32"),
         fn2_b = (weights[7].numpy()*255*255).astype("int32"),
         fn3 = (weights[8].numpy()*255).astype("int32"),
         fn3_b = (weights[9].numpy()*255*255).astype("int32"))

In [74]:
a = np.load("weights.npz")

In [75]:
w = []
for v in a:
    print(v)
    w.append(a[v].reshape(-1))
b = np.concatenate(w,0)

cov1
cov1_b
cov2
cov2_b
fn1
fn1_b
fn2
fn2_b
fn3
fn3_b


In [76]:
np.savetxt("weight.txt",b,fmt ="%d",delimiter = ",")

In [5]:
import numpy as np
#np.savetxt("image.txt",(test_images).astype("int32").reshape(-1),fmt ="%d",delimiter = ",")
np.savetxt("label.txt",(test_labels).astype("int32").reshape(-1),fmt ="%d",delimiter = ",")
print((test_images[0]).astype("int32").reshape(-1))

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  84 185 159 151  60  36   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 222 254 254 254
 254 241 198 198 198 198 198 198 198 198 170  52   

In [82]:
print((b.size-399)%400)

27


In [80]:
print(b[399:399+400])

[   0   -6   11    5   13  -27   30  -26  -71  -45  -43   -1   -9   -4
  -12  -37   -9   19  -44   17   15   57  -48  -44  -74   23    0    6
  -34   36   17  -33    9  -29  -17  -16   55    9  -43  -30    7  -91
 -130  -58   93   45   37  -28   48   26   43   23   58   27  -11   53
   10  -44  -14   69   36   27   12   31   18   52   45   28   12   74
    5   60  -39   72  -20   -8    7   40  -33   14   78   -9   49  -33
   73  -43  -11   -2   36  -69 -106  -55   38    4   -5   -7  -19  -23
  -53   18   23   -1   15  -72    6  -61  -88   -3   49  -10   51  -27
   49   22   -9  -24   53   76  -26   51  -31   59  -41   15   -2    3
  -16  -54   48   81  -79 -101  111  -38   -3    5  -10  -28   18  -34
   53   51  105  -35  -78    7   64   -4   26   34    2  -41   26  -40
  -47   78  -15   49  -12   57   -8   77  -10   -9   19   73  -40    9
   28   69  -31   39  -15   88  -77    7   92  112    8  -47   93   -6
  -84   -2  -16  -36   28  -25  -34  -14   39    1  -37  -52    3  -51
   11 

In [41]:
a = np.array([-147 , -68 ,-121  ,-59 , -30,   42])
b = np.array([40 , -29  , 20  ,-41  ,-13,   -9])
c = a * b
np.sum(c)

-3897